In [1]:
# LIBRARIES

from text_extraction import TextExtractorPixelwise
import json
import re
from typing import List, Optional
import numpy as np
from Levenshtein import distance as leve_distance
import pylcs

/home/plippmann/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LOAD ANNOTATIONS

f = open('../open-mantra-dataset/annotation.json')

annotation_data = json.load(f)

In [3]:
text_extractor = TextExtractorPixelwise(path_to_model='../Manga-Text-Segmentation/model.pkl')

2024-01-29 16:53:11.050 | INFO     | manga_ocr.ocr:__init__:13 - Loading OCR model from kha-white/manga-ocr-base
2024-01-29 16:53:12.817161: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/plippmann/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
2024-01-29 16:53:15.665 | INFO     | manga_ocr.ocr:__init__:19 - Using CUDA
2024-01-29 16:53:17.347 | INFO     | manga_ocr.ocr:__init__:32 - OCR ready
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [4]:
def remove_non_alpha(text: str) -> str:
    res = ''.join([i for i in text if i.isalpha()])
    return res

In [5]:
def find_counterpart_levenshtein(line: str, candidates: List[str]) -> Optional[str]:
    line = remove_non_alpha(line)
    distances = [leve_distance(line, remove_non_alpha(candidate)) for candidate in candidates]
    min_ind = np.argmin(distances)

    res = candidates[min_ind]
    # if distances[min_ind] <= len(line)/10:
    #     res = candidates[min_ind]
    # else:
    #     res = None

    return res, (distances[min_ind] if len(line) > 0 else 0)

In [6]:
def find_counterpart_lcs(line: str, candidates: List[str]) -> Optional[str]:
    sim = pylcs.lcs_string_of_list(line, candidates)
    if len(sim) == 0:
        return "", len(line)
    max_ind = np.argmax(sim)

    best_candidates_ind = np.flatnonzero(sim == np.max(sim))
    best_candidates = [candidates[i] for i in best_candidates_ind]
    return find_counterpart_levenshtein(line, best_candidates)

    best = candidates[max_ind]
    dist = leve_distance(remove_non_alpha(line), remove_non_alpha(best))

    return candidates[max_ind], (dist if len(remove_non_alpha(line)) > 0 else 0)

In [7]:
def score_acc_concat_lines(gt_line, ocr_line):
    gt_line = remove_non_alpha(gt_line)
    ocr_line = remove_non_alpha(ocr_line)
    if min(len(gt_line),len(ocr_line)) == 0:
        return int(len(gt_line) == len(ocr_line))
    else:
        return pylcs.lcs_sequence_length(gt_line, ocr_line)/min(len(gt_line),len(ocr_line))

In [8]:
book_titles = {
    "tojime_no_siora": 0,
    "balloon_dream": 1, 
    "tencho_isoro": 2, 
    "boureisougi": 3, 
    "rasetugari": 4, 

}

pages = annotation_data[1]['pages']

results = []

# NSFW pages return bad request so we have to skip sending those images. 
skipped_pages = [(0, 10)]

for manga_index, manga in enumerate(annotation_data):
    for page_index, page in enumerate(manga['pages']):
        image_path = "../open-mantra-dataset/" + page['image_paths']['ja'] #if (manga_index, page_index) not in skipped_pages else None
    
        ocr_lines = text_extractor.extract_lines(image_path)
        jp_ground_truth = []

        for line in page['text']:
            jp_ground_truth.append(line['text_ja'])

        jp_joined = "".join(jp_ground_truth)
        ocr_joined = "".join(ocr_lines)

        results.append((jp_joined, (ocr_joined, score_acc_concat_lines(jp_joined, ocr_joined)), (manga_index, page_index)))
       

        # translations = get_translation(ocr_lines, image_path)
            
        # print("Ground truth JP lines:")
        # for line in sorted(jp_ground_truth):
        #     print(line)

        # print("")

        # print("OCR JP lines:")
        # for line in sorted(ocr_lines):
        #     print(line)

        # for line in jp_ground_truth:
        #     results.append((line, find_counterpart_lcs(line, ocr_lines), (manga_index, page_index)))
    
    print(f"Progress: {(manga_index + 1) / len(annotation_data) * 100} %")
        

Progress: 20.0 %
Progress: 40.0 %
Progress: 60.0 %
Progress: 80.0 %
Progress: 100.0 %


In [11]:
acc_sum = 0

for (jp_gt, (ocr_line, acc), (manga_index, page_index)) in results:
    acc_sum += acc
    if acc < 0.7:
        print(f"JGT: {jp_gt}")
        print(f"OCR: {ocr_line}")
        print(f"ACC: {acc}")
        print(f"manga: {manga_index}, page: {page_index}")
        print("")

print(f"AVG_ACC = {float(acc_sum/len(results))}")

JGT: 
OCR: そして、それはそういうそういうことで、
ACC: 0
manga: 0, page: 32

JGT: 私は「生きている彼女」に会ったことはないですがさぞかし美しいお顔だったのでしょうなぜですかっ!?すずめさんお姉さんは有名な美人女優だったそうですね連日メディアが騒いでいます...私では姉のメイクをするには実力不足ということですかハハそういう訳では
OCR: お姉さんは有名な美人女優だったそうですね連日メディアが願いでいますあははっちゃーーー私は「生きている彼女」に会ったことはないですがさぞかし美しいお顔だったのでしょう姉のメイクをするには実力不足ということですかハハそういう訳ではロクへ
ACC: 0.6052631578947368
manga: 3, page: 5

JGT: ねぇ〜
OCR: 
ACC: 0
manga: 3, page: 25

JGT: ツバメさん
OCR: 
ACC: 0
manga: 3, page: 28

JGT: 心配無用十年越しの怒りそう容易くはないかっ...かかかっかかっよくぞ...
OCR: そしてフンッ心配無用！ッ十年越しの怒り．．．そう容易くはないで、オ
ACC: 0.6785714285714286
manga: 4, page: 22

JGT: ぐっ...
OCR: ．．．．．．．．．おおお！！
ACC: 0.0
manga: 4, page: 31

AVG_ACC = 0.9422587385149129


In [12]:
acc_sum = 0
significtant_errors = 0
completely_wrong = 0

for (jp_gt, (ocr_line, acc), (manga_index, page_index)) in results:
    acc_sum += acc
    if acc < 0.85:
        significtant_errors += 1
        print(f"JGT: {jp_gt}")
        print(f"OCR: {ocr_line}")
        print(f"ACC: {acc}")
        print(f"manga: {manga_index}, page: {page_index}")
        print("")

    if acc < 0.7 >= min(len(remove_non_alpha(jp_gt)), len(remove_non_alpha(ocr_line))):
        completely_wrong += 1        

print(f"AVG_ACC = {float(acc_sum/len(results))}")

print(f"total_number_of_lines: {len(results)}")

print(f"significant errors: {significtant_errors}")

print(f"completely_wrong: {completely_wrong}")


JGT: あ?ててっ思ったより窓が高かった...おい...なんだこいつ...シオラッ!?いやぁ夜分遅くにすいません...
OCR: いや、それは．．．あ？ブッおい．．なんだこいつ．．．シオラッ！？いやぁ夜分遅くにすいません．．．
ACC: 0.7878787878787878
manga: 0, page: 21

JGT: 
OCR: そして、それはそういうそういうことで、
ACC: 0
manga: 0, page: 32

JGT: バルーンドリーム朽鷹みつき
OCR: ンドリーム朽鷹みつきバルー
ACC: 0.7692307692307693
manga: 1, page: 2

JGT: 死んでるんですよねぇようこそ
亡霊葬儀屋さん
吉良いと「視える」人間ってヤツでしてえ?亡くなったお姉さんのお化粧を?烏丸葬儀社社長
烏丸枢(からすまくるる）　２５歳はい...葬儀依頼人
桜野　すずめ　１９歳
OCR: 死んでるんですよねぇ視える」人間ってヤツでしてようこそ亡霊葬儀屋さん古いとふぅううんっ亡くなったお姉さんのお化粧を？鳥丸葬儀を社長歳丸。枢（からすまーくるる）２歳の葬儀依頼人桜野すずめ１９歳
ACC: 0.7710843373493976
manga: 3, page: 3

JGT: 私は「生きている彼女」に会ったことはないですがさぞかし美しいお顔だったのでしょうなぜですかっ!?すずめさんお姉さんは有名な美人女優だったそうですね連日メディアが騒いでいます...私では姉のメイクをするには実力不足ということですかハハそういう訳では
OCR: お姉さんは有名な美人女優だったそうですね連日メディアが願いでいますあははっちゃーーー私は「生きている彼女」に会ったことはないですがさぞかし美しいお顔だったのでしょう姉のメイクをするには実力不足ということですかハハそういう訳ではロクへ
ACC: 0.6052631578947368
manga: 3, page: 5

JGT: お姉さんの事故後のお顔ご覧になりました?それはそうでしょうねぇ...いえ病院の方からあまり見ない方がいいと言われて...今の桜野ツバメさんは貴女の知っているお顔じゃありませんから
OCR: お姉さんの事故後のお顔こんなことがあったのですから、そのままこれはそういうことをしたらしいんだけど．．．あまり

NameError: name 'dist_sum' is not defined